In [24]:
import numpy as np
from   sklearn.model_selection import train_test_split
import tensorflow as tf
from   tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D)
from   tensorflow.keras.losses import categorical_crossentropy as cxe

In [11]:
!ls ../../../../data/quickdraw

full_numpy_bitmap_ambulance.npy   full_numpy_bitmap_mermaid.npy
full_numpy_bitmap_crocodile.npy   full_numpy_bitmap_raccoon.npy
full_numpy_bitmap_eye.npy         full_numpy_bitmap_rifle.npy
full_numpy_bitmap_flamingo.npy    full_numpy_bitmap_snail.npy
full_numpy_bitmap_harp.npy        full_numpy_bitmap_stethoscope.npy


In [12]:
DATA = '../../../../data/quickdraw'
FILE_TEMPLATE = f'{DATA}/full_numpy_bitmap_%s.npy'
categories = ['ambulance', 'crocodile', 'eye', 'flamingo', 'harp', 
              'mermaid', 'raccoon', 'rifle', 'snail', 'stethoscope']
filenames = [FILE_TEMPLATE % x for x in categories]

In [13]:
BATCH = 128
DIM = 28
N_IMAGES = 100000 # reduce if mem issues
N_FILES = len(categories)
IMAGES_PER_CATEGORY = N_IMAGES // N_FILES
IMAGES_PER_CATEGORY

10000

### Preprocessing

In [14]:
i = 0
for path in filenames:
    x = np.load(path)
    x = x.astype('float32') / 255.
    y = [i] * len(x)
    x = x[:IMAGES_PER_CATEGORY]
    y = y[:IMAGES_PER_CATEGORY]
    if i == 0:
        x_all = x
        y_all = y
    else:
        x_all = np.concatenate((x, x_all), axis=0)
        y_all = np.concatenate((y, y_all), axis=0)
    i += 1

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=1103)

In [18]:
X_train = X_train.reshape(X_train.shape[0], DIM, DIM, 1)
X_test = X_test.reshape(X_test.shape[0], DIM, DIM, 1)
input_shape = (DIM, DIM, 1)

In [19]:
y_train = tf.keras.utils.to_categorical(y_train, N_FILES)
y_test = tf.keras.utils.to_categorical(y_test, N_FILES)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1103)

### ConvNet model

In [22]:
DROP = 0.25

In [23]:
mod = tf.keras.Sequential()

mod.add(Conv2D(
    32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
mod.add(MaxPooling2D(pool_size=(2, 2)))
mod.add(Dropout(DROP))

mod.add(Conv2D(64, (3, 3), activation='relu'))
mod.add(MaxPooling2D(pool_size=(2, 2)))
mod.add(Dropout(DROP))

mod.add(Flatten())
mod.add(Dense(128, activation='relu'))
mod.add(Dropout(2 * DROP))
mod.add(Dense(N_FILES, activation='softmax'))

In [25]:
mod.compile(loss=cxe, 
            optimizer=tf.keras.optimizers.Adadelta(), 
            metrics=['accuracy'])

In [26]:
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0